<a href="https://colab.research.google.com/github/MANDY-QIU/language-models/blob/master/Copy_of_ner_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [2]:
!pip install --upgrade spacy==2.2.4 allennlp==0.9.0
import spacy
print(spacy.__version__)

Requirement already up-to-date: spacy==2.2.4 in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 7.6MB 6.1MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 266kB 46.4MB/s 
     |████████████████████████████████| 163kB 47.6MB/s 
     |████████████████████████████████| 133kB 49.7MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 317kB 55.1MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 245kB 51.2MB/s 
     |████████████████████████████████| 5.3MB 52.3MB/s 
     |████████████████████████████████| 133kB 52.1MB/s 
     |████████████████████████████████| 1.1MB 54.1MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 245kB 49.0MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 7.0MB 46.4MB/s 
  Created wheel fo

In [3]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [4]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [6]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

562it [00:00, 16506.07it/s]
23it [00:00, 7915.73it/s]
100%|██████████| 585/585 [00:00<00:00, 48144.17it/s]


## Define and Train Model

In [7]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9142 ||: 100%|██████████| 281/281 [00:01<00:00, 186.49it/s]
accuracy: 0.7878, loss: 1.1933 ||: 100%|██████████| 12/12 [00:00<00:00, 386.09it/s]
accuracy: 0.8442, loss: 0.7294 ||: 100%|██████████| 281/281 [00:01<00:00, 216.26it/s]
accuracy: 0.7878, loss: 1.1755 ||: 100%|██████████| 12/12 [00:00<00:00, 416.77it/s]
accuracy: 0.8442, loss: 0.7164 ||: 100%|██████████| 281/281 [00:01<00:00, 214.26it/s]
accuracy: 0.7878, loss: 1.1787 ||: 100%|██████████| 12/12 [00:00<00:00, 415.40it/s]
accuracy: 0.8442, loss: 0.7071 ||: 100%|██████████| 281/281 [00:01<00:00, 186.22it/s]
accuracy: 0.7878, loss: 1.1674 ||: 100%|██████████| 12/12 [00:00<00:00, 353.10it/s]
accuracy: 0.8442, loss: 0.6989 ||: 100%|██████████| 281/281 [00:01<00:00, 216.76it/s]
accuracy: 0.7878, loss: 1.1713 ||: 100%|██████████| 12/12 [00:00<00:00, 371.06it/s]
accuracy: 0.8442, loss: 0.6910 ||: 100%|██████████| 281/281 [00:01<00:00, 215.92it/s]
accuracy: 0.7878, loss: 1.1569 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 98,
 'best_validation_accuracy': 0.8653061224489796,
 'best_validation_loss': 0.44252253993666574,
 'epoch': 99,
 'peak_cpu_memory_MB': 3826.724,
 'peak_gpu_0_memory_MB': 957,
 'training_accuracy': 0.9186526850969177,
 'training_cpu_memory_MB': 3826.724,
 'training_duration': '0:02:23.643389',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 957,
 'training_loss': 0.21970394746258728,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8612244897959184,
 'validation_loss': 0.4620858648753104}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [33]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]

In [34]:
#predictor = SentenceTaggerPredictor(model, dataset_reader=PosDatasetReader())
tag_logits = tag_sentence(predictor.predict_instance(s)['tag_logits'])
print(tag_logits)
tag_ids = np.argmax(tag_logits, axis=-1)
#print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
    
    #print(vocab.get_index_to_token_vocabulary('labels'))
    #shutil.rmtree(serialization_dir)

NameError: ignored

In [24]:
len(tag_logits)

5

In [37]:
len(baseline_output)

23

In [12]:
def violations(tagged):
  Raw_l=[]
  Predict_l=[]
  for i in tagged:
    raw_l=[j[1] for j in i]
    Raw_l.append(raw_l)
    predict_l = [j[2] for j in i]
    Predict_l.append(predict_l)
  Count=0
  for i in Predict_l:
    split_l = [j.split("-") for j in i]
    count = 0
    for j in range(len(split_l)):
      if len(split_l[j])==2:
        #error type begining no "B"
        if j==0 and split_l[j][0]!='B':
          count = count+1
        #error type ending solo "I":
        if j==len(split_l)-1 and j[0]=='I' and split_l[j][1]!=split_l[j-1][1]:
          count = count+1
        if j>0 and j<len(split_l)-1:
          #error 2 B
          if split_l[j][0]=='B' and len(split_l[j-1])==2 and split_l[j-1][0]=='B' and split_l[j][1]==split_l[j-1][1]:
            count = count+1
          #error b-x i-y
          if split_l[j][0]=='I' and len(split_l[j-1])==2 and split_l[j-1][0]=='B' and split_l[j][1]!=split_l[j-1][1]:
            count = count+1
          #error o i-x
          if split_l[j][0]=='I' and len(split_l[j-1])==1:
            count = count+1
          #error i-x i-y
          if split_l[j][0]=='I' and len(split_l[j-1])==2 and split_l[j-1][0]=='I' and split_l[j][1]!=split_l[j-1][1]:
            count = count+1
    # print(i)
    # print(count)
    # print("*******************************")
    Count = Count+count
  return Count

In [14]:
Count=violations(baseline_output)
Count

35

In [17]:
#baseline_output

In [76]:
# get the last element of each tuple 
third_ele = []

for item in baseline_output:
  phase=[]
  for a_tuple in item:
    phase.append(a_tuple[1])
  third_ele.append(phase)
print(third_ele)


[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'B-CARDINAL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'O'], ['B-TIME', 'I-TIME', 'O', 'B-PERSON', 'I-PERSON', 'B-ORDINAL', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O'], ['O', 'O', 'O', 'B-CARDINAL', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CARDINAL', 'I-CARDINAL', 'I-CARDINAL', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CARDINAL', 'I-CARDINAL', 'I-CARDINAL', 'I-CARDINAL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-CARDINAL', 'I-CARDINAL', 'I-CARDINAL', 'O', 'O', 'O', 'O'], ['B-EVENT', 'I-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O',

In [10]:
from seqeval.metrics import f1_score
from seqeval.metrics import precision_score
from seqeval.metrics import accuracy_score
from seqeval.metrics import recall_score
from seqeval.metrics import classification_report


In [104]:
def violations(tagged):
  third_ele = []
  for item in tagged:
     phase=[]
     for a_tuple in item:
       phase.append(a_tuple[2])
     third_ele.append(phase)
  count = 0
  for l in third_ele:
    if l[0][0] == 'I':
      count += 1
    for pair in zip(l, l[1:]):
      if (pair[0][0], pair[1][0]) == ('O', 'I'):
        count += 1
      if (pair[0][0], pair[1][0]) == ('B', 'I') and pair[0][1:] != pair[1][1:]:
        count += 1
      if (pair[0][0], pair[1][0]) == ('I', 'I') and pair[0] != pair[1]:
        count += 1

  return count

count=violations(baseline_output)
print("number of violation:", count)
#return the span-level precision, recall, and F1
def span_stats(tagged):
  second_ele = []
  for item in tagged:
    phase=[]
    for a_tuple in item:
      phase.append(a_tuple[1])
    second_ele.append(phase)
  third_ele = []
  for item in tagged:
     phase=[]
     for a_tuple in item:
       phase.append(a_tuple[2])
     third_ele.append(phase)

  y_true = second_ele
  y_pred = third_ele
  precision=precision_score(y_true, y_pred)
  recall=recall_score(y_true, y_pred)
  f1=f1_score(y_true, y_pred)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}
#print recall, F1, Precision
span_stats(baseline_output)

number of violation: 39


{'f1': 0.16666666666666669,
 'precision': 0.13846153846153847,
 'recall': 0.20930232558139536}

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [113]:
# TODO: count the number of NER label violations,

# such as O followed by I-TYPE or B-TYPE followed by I-OTHER_TYPE

#o-i, B-PER, I-LOC, 
#BOI, B-PER, IS-O, B-PERSON, I-PERSON, B-LOC, I-LOC, BOISE, 

# Take tagger output as input
def violations(tagged):
  third_ele = []
  for item in tagged:
     phase=[]
     for a_tuple in item:
       phase.append(a_tuple[2])
     third_ele.append(phase)
  count = 0
  for l in third_ele:
    if l[0][0] == 'I':
      count += 1
    for pair in zip(l, l[1:]):
      if (pair[0][0], pair[1][0]) == ('O', 'I'):
        count += 1
      if (pair[0][0], pair[1][0]) == ('B', 'I') and pair[0][1:] != pair[1][1:]:
        count += 1
      if (pair[0][0], pair[1][0]) == ('I', 'I') and pair[0] != pair[1]:
        count += 1

  return count

count=violations(baseline_output)
print("number of violation:", count)
# TODO: return the span-level precision, recall, and F1

# Only count valid spans that start with a B tag, followed by zero or more I tags of the same type.


# Take tagger output as input
def span_stats(tagged):
  second_ele = []
  for item in tagged:
    phase=[]
    for a_tuple in item:
      phase.append(a_tuple[1])
    second_ele.append(phase)
  third_ele = []
  for item in tagged:
     phase=[]
     for a_tuple in item:
       phase.append(a_tuple[2])
     third_ele.append(phase)
  for i in range(len(third_ele)):
    l = third_ele[i]
    m = second_ele[i]
    num = len(l)
    j = 1
    # remove_index = []
    # while j < num:
    #     if (l[j-1][0], l[j][0]) == ('O', 'I'):
    #         remove_index.append(j)
    #         j += 1
    #         while j < num:
    #             if m[j][0] == 'I':
    #                 remove_index.append(j)
    #                 j += 1
    #             else:
    #                 break
    #     else:
    #         j += 1
    # for index in sorted(remove_index, reverse=True):
    #     del third_ele[i][index]
    #     del second_ele[i][index]
  y_true = second_ele
  y_pred = third_ele
  precision=precision_score(y_true, y_pred)
  recall=recall_score(y_true, y_pred)
  f1=f1_score(y_true, y_pred)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}
#print recall, F1, Precision
span_stats(baseline_output)

number of violation: 39


{'f1': 0.18604651162790697,
 'precision': 0.18181818181818182,
 'recall': 0.19047619047619047}

In [102]:
# the number of entites in pred
# def span_stats(tagged):
#   third_ele = []
#   for item in baseline_output:
#      phase=[]
#      for a_tuple in item:
#        phase.append(a_tuple[2])
#        third_ele.append(phase)
#   return third_ele
# def span(tagged):
#   count=0
#   for l in third_ele:
#         for i in l:
#             if i[0] =='B':
#                 count += 1
#   return count
# count=span(third_ele)
# count

In [31]:
from seqeval.metrics import f1_score
from seqeval.metrics import precision_score
from seqeval.metrics import accuracy_score
from seqeval.metrics import recall_score
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

In [36]:
def span_stats(tagged):
  second_ele = []
  for item in tagged:
    phase=[]
    for a_tuple in item:
      phase.append(a_tuple[1])
    second_ele.append(phase)
  third_ele = []
  for item in tagged:
     phase=[]
     for a_tuple in item:
       phase.append(a_tuple[2])
     third_ele.append(phase)

  y_true = second_ele
  y_pred = third_ele
  precision=precision_score(y_true, y_pred)
  recall=recall_score(y_true, y_pred)
  f1=f1_score(y_true, y_pred)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}

In [90]:
span_stats(baseline_output)

{'f1': 0.24761904761904757,
 'precision': 0.20967741935483872,
 'recall': 0.3023255813953488}

## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [115]:
# This code show how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))

data=vocab.get_index_to_token_vocabulary('labels')

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [18]:
tag_log=LstmTagger().forward(baseline_output)
#score=tag_sentence(baseline_output)

TypeError: ignored

34

In [ ]:
transitions = ['A', 'B', 'B', 'C', 'B', 'A', 'D', 'D', 'A', 'B', 'A', 'D']

def rank(c):
    return ord(c) - ord('A')

T = [rank(c) for c in transitions]